In [158]:
import os
import cv2
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
import matplotlib.pyplot as plt
import pytesseract
import re
import pandas as pd

# Function to display image using matplotlib
def show_image(img, title="Image"):
    plt.figure(figsize=(10, 6))
    plt.imshow(img, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

def process_image_turn(image_path, file):
    img = Image.open(image_path)
    
       # padding 해줘서 돌릴 때 짤리지 않게
    img = ImageOps.expand(img, border=30, fill='white')
    
    img = np.array(img)  # Convert the PIL image to a NumPy array
    #show_image(img)
    ##################### rotate
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply edge detection
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    # Find lines using HoughLines
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)

    if lines is None:
        return pd.DataFrame(columns=['ID', 'OCR', 'target'])

    # Calculate the angle of the skew
    angles = []
    for line in lines:
        rho, theta = line[0]
        angle = np.degrees(theta) - 90
        angles.append(angle)


    # Compute the median angle
    median_angle = np.median(angles)

    # Rotate the image to correct the skew if necessary
    (h, w) = img.shape[:2]
    center = (w // 2, h // 2)

    #print(median_angle)
    if abs(median_angle) > 12.0:  # Adjust this threshold as necessary
        M = cv2.getRotationMatrix2D(center, median_angle, 1.0)
        rotated = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    else:
        rotated = img  # Use the original image if no rotation is needed

    

    all_results = []

    # Convert back to PIL image to display
    rotated_pil = Image.fromarray(cv2.cvtColor(rotated, cv2.COLOR_BGR2RGB))
    
    
    
    gray_image = cv2.cvtColor(rotated, cv2.COLOR_BGR2GRAY)
    average_brightness = np.mean(gray_image)
    #  print(average_brightness)
    if average_brightness<160:
        # 밝기 조절
        enhancer = ImageEnhance.Brightness(rotated_pil)
        rotated_pil = enhancer.enhance(2.0)  # 밝기를 증가, 이것보다 높게 하면 자동회전이 잘못되서 위치 바꿈

        # 대비 조절
        enhancer = ImageEnhance.Contrast(rotated_pil)
        rotated_pil = enhancer.enhance(3.0)  # 대비를 증가, 이것보다 높게 하면 자동회전이 잘못되서 위치 바꿈
    

    def process_and_append(image, description):
        #print(description)
        #show_image(image)
        df = process_image_with_ocr(image, file)
        #print(df['OCR'])
        if not df.empty:
            print('*****************',file)
            # print(description, df['OCR'], df['target'])
            all_results.append(df)
            return True
        return False

    if process_and_append(rotated_pil, "rotated_pil"):
        return all_results
    
    rotated_270 = rotated_pil.rotate(270, expand=True)
    if process_and_append(rotated_270, "rotated_270"):
        return all_results
    
    rotated_180 = rotated_pil.rotate(180, expand=True)
    if process_and_append(rotated_180, "rotated_180"):
        return all_results
    
    rotated_90 = rotated_pil.rotate(90, expand=True)
    if process_and_append(rotated_90, "rotated_90"):
        return all_results
    
    

    mirrored_pil1 = ImageOps.mirror(rotated_pil)
    if process_and_append(mirrored_pil1, "mirrored_pil1"):
        return all_results
    
    mirrored_rotated_270 = mirrored_pil1.rotate(270, expand=True)
    if process_and_append(mirrored_rotated_270, "mirrored_rotated_270"):
        return all_results
    
    mirrored_rotated_180 = mirrored_pil1.rotate(180, expand=True)
    if process_and_append(mirrored_rotated_180, "mirrored_rotated_180"):
        return all_results
    
    mirrored_rotated_90 = mirrored_pil1.rotate(90, expand=True)
    if process_and_append(mirrored_rotated_90, "mirrored_rotated_90"):
        return all_results


    return all_results

# Function to process an image with OCR
def process_image_with_ocr(img, image_name):
    if img is None:
        return pd.DataFrame(columns=['ID', 'OCR', 'target'])

    img = np.array(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.fastNlMeansDenoising(gray, h=3)
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    kernel = np.ones((5, 5), np.uint8)
    dilated = cv2.dilate(binary, kernel, iterations=1)
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    x_min = x
    y_min = y
    x_max = x + w
    y_max = y + h
    padding = 50
    y_min = max(0, y_min - padding)
    y_max = min(gray.shape[0], y_max + padding)
    cropped_img = gray[y_min:y_max, x_min:x_max]
    enhancer = ImageEnhance.Contrast(Image.fromarray(cropped_img))
    enhanced_img = enhancer.enhance(2)
    enhanced_img = enhanced_img.filter(ImageFilter.SHARPEN)
    # enhanced_img = enhanced_img.resize((enhanced_img.width * 2, enhanced_img.height * 2), Image.BICUBIC)
    #show_image(enhanced_img)

    custom_config = r'--oem 3 --psm 6'
    text = pytesseract.image_to_string(enhanced_img, lang='kor', config=custom_config)
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.replace(" ", "")
    #print(text)
    
    text2 = pytesseract.image_to_string(gray, lang='kor', config=custom_config)
    text2 = re.sub(r'\s+', ' ', text2).strip()
    text2 = text2.replace(" ", "")
    #print(text2)
    
    
    
    # 7,3 에 대해서만 판단하는 것이 맞는 듯. submission 94% 정확도가 올라가면 pytesseract ocr 정확도가 따라가지 못함.
    keywords_4 = ['진단서','전단서']
    keywords_5= ['운전면허증']
    keywords_7 = ['통원','동원']
    keywords_10 = ['납입','남입','남임','남원','입확인서','임확인서','입확원서','임확원서']
    keywords_3 = ['입원','퇴원','원확인서','완확원서','완확원서']
    keywords_8= ['주민등록증','주민등록중']
    keywords_9= ['대한민국','여권']    
    keywords_11= ['영수증','복약','조제약']
    keywords_13= ['이력서','자소서']
    keywords_14 = ['소견서']
    keywords_15= ['자동차등록증','자동차등록중','자동자등록증']
    keywords_12= ['처방전','처방']
    keywords_1= ['건강보험','임신','출산']
    keywords_6 = ['외래', '계산서']
    
    keywords_dict = {
        
        # 순서가 중요!!!! ocr에 검출되는 단어들이 중복되서 나오기 때문.
        4: keywords_4,
        5: keywords_5,
        
        7: keywords_7,
        10: keywords_10,
        3: keywords_3,
        8: keywords_8,
        9: keywords_9,
        
        11: keywords_11,
        
        13: keywords_13,
        14: keywords_14,
        15: keywords_15,
        12: keywords_12,
        1: keywords_1,
        6: keywords_6,
    }

    results = []
    for number, keywords in keywords_dict.items():
        if any(keyword in text for keyword in keywords):
            results.append((image_name, text, number))
            break
        elif any(keyword in text2 for keyword in keywords):
            results.append((image_name, text2, number))
            break

    df = pd.DataFrame(results, columns=['ID', 'OCR', 'target'])
    return df

pd.set_option('display.max_colwidth', None)

# Function to process all images in a directory
def process_directory(directory):
    all_results = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(root, file)
                results = process_image_turn(image_path, file)
                if len(results) > 0:
                    all_results.extend(results)
                              
    if all_results:
        final_df = pd.concat(all_results, ignore_index=True)
    else:
        final_df = pd.DataFrame(columns=['ID', 'OCR', 'target'])
    return final_df



# # Directory containing the images
# directory = '/data/ephemeral/home/datasets_fin/test'

# # Process all images in the directory
# result_df = process_directory(directory)

# # Save the result to a CSV file
# result_df.to_csv(r'/data/ephemeral/home/notebook/ocr_output_3_7.csv', index=False)

# # Display the final result DataFrame
# print("Final Result DataFrame:")
# print(result_df)




output = pd.read_csv(r'/data/ephemeral/home/notebook/output.csv')
filtered_ids = output[(output['target'].isin([3,7]))]['ID']
# print(len(filtered_ids))



# 디렉토리 내 이미지를 처리
def process_directory2(directory, valid_ids):
    all_results = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file in valid_ids.values:
                image_path = os.path.join(root, file)
                # print("Processing:", image_path)  # 경로 확인용 출력
                results = process_image_turn(image_path, file)  # 각 파일에 대해 이미지 처리 함수 호출
                if len(results) > 0:
                    all_results.extend(results)  # 결과 합치기

    if all_results:
        final_df = pd.concat(all_results, ignore_index=True)
    else:
        final_df = pd.DataFrame(columns=['ID', 'OCR', 'target'])
    return final_df


# 이미지 처리
directory = '/data/ephemeral/home/datasets_fin/test'
final_df = process_directory2(directory, filtered_ids)
final_df.to_csv(r'/data/ephemeral/home/notebook/ocr_output_3_7.csv', index=False)

print(final_df)


***************** 8538d5326f92f9fb.jpg
***************** f95c84c901644f43.jpg
***************** b2a1a2dc8491cd35.jpg
***************** 1efca93753690930.jpg
***************** fc51e2f2a0717743.jpg
***************** cb3179f4bfdf1878.jpg
***************** d2a9e0d2faa42d67.jpg
***************** e24bc8764e665e39.jpg
***************** 5d778edcf3564010.jpg
***************** 2d075b49d57a56c7.jpg
***************** 00e15da96484eb94.jpg
***************** 8f06b90c1c5e79f4.jpg
***************** bddaf431af26a850.jpg
***************** 5f2ea6fb8d52e684.jpg
***************** ebae9fff20813575.jpg
***************** 6d31a57776ef3b85.jpg
***************** dccbcea4a69ce6df.jpg
***************** 8cf431a1ea0be8b2.jpg
***************** 725a6d2df73c9fad.jpg
***************** 44db33edab62fbab.jpg
***************** c4b25435ed6e5ea7.jpg
***************** 6f6e825853bbccad.jpg
***************** 5fc2b023b04011dd.jpg
***************** 56a8a6edde03bb05.jpg
***************** acbdc2456b62632f.jpg
***************** e5aa2a6